Pull some HTML from wayback machine and store it


In [ ]:
var importer = require('../Core');
var util = require('util');
var request = util.promisify(require('request'));
var {JSDOM} = require('jsdom');
var fs = require('fs');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Collections/flagstaff-happy';


var locations, getAllXPath;
function getLocations() {
    if(typeof locations !== 'undefined') {
        return Promise.resolve(locations);
    }
    return request('https://web.archive.org/web/20160322001433/http://keepflaghappy.com/happy-hours/friday/')
        .then(r => {
            const getAllXPath = importer.import('all elements xpath from string')(r.body);
            return getAllXPath([
                '//a[contains(@href, "by-location")]/@href',
            ]);
        })
        .then(r => {
            locations = r
                .filter(l => l.match(/by-location\/[^#]+/ig))
                .map(l => l.includes('archive.org') ? l : 'https://web.archive.org' + l);
            return locations;
        });
}

function getLocation(l) {
    console.log('Downloading ' + l);
    return request(l)
        .then(r => {
            const getAllXPath = importer.import('all elements xpath from string')(r.body);
            return getAllXPath([
                '//h3[contains(., "Mon") or contains(., "Tue") or contains(., "Wed") or contains(., "Thu") or contains(., "Fri") or contains(., "Sat") or contains(., "Sun")]',
                {
                    dotw: './text()',
                    time: './following-sibling::p[count(./preceding-sibling::h3)=$i+1]//text()',
                    deals: './following-sibling::ul[count(./preceding-sibling::h3)=$i+1]//text()'
                }
            ]);
        })
        .then(r => {
            const happy = r.map(l => {
                return {
                    dotw: l.dotw,
                    time: l.time,
                    deals: l.deals.join('\n').trim().split(/\s*\n+\s*/ig)
                };
            });
            const name = l.trim().replace(/\/$/ig, '').split('/').pop().replace(/[^a-z0-9-_]/ig, '_');
            fs.writeFileSync(path.join(project, name + '.json'), JSON.stringify(happy, null, 4));
            return happy;
        })
}

$$.async();
getLocations()
    .then(r => {
        return importer.runAllPromises(r.map(l => resolve => {
            return getLocation(l)
                .then(r => setTimeout(() => resolve(r), 100))
        }));
    })
    .then(r => $$.mime({'text/plain': JSON.stringify(r, null, 4)}))
    .catch(e => $$.sendError(e))



Gather resturant data from google maps?

google maps data list?


In [2]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');
var fs = require('fs');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Collections/flagstaff-happy';

function getResultsPage() {
    return client
        .then(() => getAllXPath([
            '//*[contains(@class, "section-result-text-content")]',
            {
                name: './/h3[contains(@class, "section-result-title")]//text()',
                description: './/span[contains(@class, "section-result-details")]//text()|.//span[contains(@class, "section-result-location")]//text()'
            }
        ]))
        .then(r => r.map(l => ({
            name: typeof l.name === 'string' ? l.name : l.name.join('\n').trim(),
            description: typeof l.description === 'string'
                ? l.description
                : l.description.join('\n').trim().split(/\s*\n\s*/ig),
        })));
}

function getAllResults() {
    var locations = [];
    return getResultsPage()
        .then(newLocs => {
            locations = newLocs;
            return client.isExisting('//*[contains(@class, "section-pagination-right")]//button[contains(@aria-label, "Next page") and not(@disabled)]');
        })
        .then(is => {
            if(is) {
                return client.click('//*[contains(@class, "section-pagination-right")]//button[contains(@aria-label, "Next page") and not(@disabled)]')
                    .pause(3000)
                    .then(() => getAllResults())
                    .then(newLocs => locations.concat(newLocs))
            } else {
                return locations;
            }
        });
}

function getNearbyJSON(place) {
    if(typeof place === 'undefined') {
        place = 'bars+near+Flagstaff,+AZ';
    }
    return client.url('https://www.google.com/maps/search/' + place)
        .then(() => getAllResults())
        .then(r => {
            const day = new Date();
            const date = day.getFullYear() + '-' + (day.getMonth() + 1) + '-' + day.getDate();
            fs.writeFileSync(path.join(project, 'locations-' + date + '.json'), JSON.stringify(r, null, 4));
            return r;
        })
}
module.exports = getNearbyJSON;

if(typeof $$ !== 'undefined') {
    $$.async();
    runSeleniumCell('google maps data list', false)
        .then(func => func())
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


reading notebook /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb[0]
reading notebook /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[0]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[3]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[4]
reading notebook /Users/briancullinan/jupytangular2/Utilities/file system.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[0]
reading notebook /Users/briancullinan/jupytangular2/Analytics/budgeting.ipynb
reading notebook /Users/briancullinan/jupytangular2/Analytics/code analysis.ipynb
reading notebook /Users/briancullina

reading notebook /Users/briancullinan/jupytangular2/Utilities/levenshtein.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/mono.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/nmap.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/notebook.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/npm.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/openssl.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/parse.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/polyfills.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/registry.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/scraping.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/sql to json.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/uuid.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/vnc.ipynb
reading notebook /

[ { name: 'Hops On Birch',
    description: [ 'Bar', '22 E Birch Ave #2' ] },
  { name: 'Monte Vista Cocktail Lounge',
    description: [ 'Cocktail Bar', '100 N San Francisco St' ] },
  { name: 'The State Bar',
    description: [ 'Bar', '10 Historic Rte 66 E' ] },
  { name: 'Collins Irish Pub & Grill',
    description: [ 'Grill', '5536, 2 N Leroux St' ] },
  { name: 'The McMillan Bar and Kitchen',
    description: [ 'Bar', '2 West Route 66' ] },
  { name: 'The Annex Cocktail Lounge',
    description: [ 'Cocktail Bar', '50 S San Francisco St' ] },
  { name: 'The Mayor',
    description: [ 'Grill', '409 S San Francisco St' ] },
  { name: 'The Green Room',
    description: [ 'Bar', '15 N Agassiz St' ] },
  { name: 'Shift Kitchen & Bar',
    description: [ 'Gastropub', '107 N San Francisco St #2' ] },
  { name: 'Majerle\'s Sports Grill',
    description: [ 'Sports Bar', '102 Historic Rte 66' ] },
  { name: 'Beaver Street Brewery',
    description: [ 'Brewpub', '11 S Beaver St #1' ] },
  { 

convert location names to google maps locations with happy hour data


In [ ]:
var importer = require('../Core');
var fs = require('fs');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Collections/flagstaff-happy';

function loadLocations() {
    const day = new Date();
    const date = day.getFullYear() + '-' + (day.getMonth() + 1) + '-' + day.getDate();
    return JSON.parse(fs.readFileSync(path.join(project, 'locations-' + date + '.json')).toString());
}



if(typeof $$ !== 'undefined') {
    $$.async();
    loadLocations()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}
